# **Motown Mo’ Buses!**
## *Effects of Historic Redlining on Transit Access and Employment in Motown*
Amelia Garcia (agg76), Abduljellil Hamid (ahh87), Dominic Martinelli (ddm94), Ben Perl (bdp58), Izzy Prager (ikp5)

# Dependencies, Configs, Imports, Datasets

## Configs

In [1]:
%config InlineBackend.figure_formats = ["retina"]

## Packages/Dependencies

In [ ]:
!pip install -q osmnx descartes scikit-learn seaborn census pandas requests matplotlib geopandas gdown selenium folium

# Download the motor businesses file from Google Drive
#!gdown --id 1ylPscnouYGm7STGO1kyx0n6QvxQZF3xc --output motor_businesses_all_f.csv

## Imports

In [ ]:
import numpy as np
import pandas as pd
import geopandas as gpd
import seaborn as sns
import folium
import requests
import io
from PIL import Image
import selenium

from matplotlib import pyplot as plt
from matplotlib import lines as mlines
from folium import GeoJson
from shapely.geometry import mapping, Point, LineString, Polygon

import networkx as nx
import osmnx as ox

from tqdm import tqdm
from descartes import PolygonPatch
from joblib import Parallel, delayed

from branca.utilities import color_brewer
from census import Census

from shapely.ops import unary_union

from sklearn.linear_model import LinearRegression

## API Calls/Dataset Reads

In [ ]:
# Detroit DoT Bus Routes
ddot_routes_url = "https://services2.arcgis.com/qvkbeam7Wirps6zC/arcgis/rest/services/DDOT_Bus_Routes/FeatureServer/0/query?outFields=*&where=1%3D1&f=geojson"
ddot_stops_url = "https://services2.arcgis.com/qvkbeam7Wirps6zC/arcgis/rest/services/DDOT_Bus_Stops/FeatureServer/0/query?outFields=*&where=1%3D1&f=geojson"

ddot_routes = gpd.read_file(ddot_routes_url)
ddot_stops = gpd.read_file(ddot_stops_url)

counties=["099","125","163"]

!curl -O -L https://www2.census.gov/geo/tiger/TIGER2022/TRACT/tl_2022_26_tract.zip

!unzip tl_2022_26_tract.zip

michigan_tract_gdf=gpd.read_file("tl_2022_26_tract.shp")

#mi_tract_gdf = gpd.read_file("https://www2.census.gov/geo/tiger/TIGER2020/TRACT/tl_2020_26_tract.zip").set_index("GEOID").to_crs("EPSG:2163")
detroit_tract_gdf = michigan_tract_gdf[michigan_tract_gdf.STATEFP == "26"]
detroit_tract_gdf = detroit_tract_gdf[detroit_tract_gdf.COUNTYFP.isin(counties)]
wayne_tract_gdf = michigan_tract_gdf[michigan_tract_gdf.STATEFP == "26"]
wayne_tract_gdf = wayne_tract_gdf[wayne_tract_gdf.COUNTYFP.isin(counties)]
detroit_outline = gpd.read_file("https://services2.arcgis.com/qvkbeam7Wirps6zC/arcgis/rest/services/City_of_Detroit_Boundary/FeatureServer/0/query?outFields=*&where=1%3D1&f=geojson")

census = Census("", year=2020)
areas_gdf = gpd.read_file("https://dsl.richmond.edu/panorama/redlining/static/mappinginequality.gpkg")
hri_gdf = gpd.read_file("https://raw.githubusercontent.com/PUBPOL-2130/notebooks/refs/heads/main/data/HRI2020.zip")

# read the CSV file
!curl -L -o motor_businesses_all_f.csv.zip https://github.com/PUBPOL-2130/projects/raw/main/R1/motor_businesses_all_f.csv.zip
!unzip -o motor_businesses_all_f.csv.zip -d motor_data
motor_businesses = pd.read_csv("motor_data/motor_businesses_all_f.csv")

#Redlining in Detroit

In [ ]:

#looking at Detroit redlining
detroit_areas_gdf = areas_gdf[(areas_gdf["city"] == "Detroit")].copy() #select for Detroit within larger gdf
grade_to_score = {"A": 4, "B": 3, "C": 2, "D": 1} #assign numerical values to redlining scores
detroit_areas_gdf["score"] = detroit_areas_gdf["grade"].map(grade_to_score)

#create  visual with updated Detroit coordinates
detroit_hri_map = folium.Map([42.3314, -83.0458], zoom_start=10, tiles="CartoDB positron") #changed basemap to make background less busy
detroit_chloro = folium.Choropleth( #create chloropleth!
    geo_data=detroit_areas_gdf,
    name="choropleth",
    data=detroit_areas_gdf,
    columns=["area_id", "score"],
    key_on="feature.properties.area_id",
    fill_color="OrRd_r", #orange to red gradient to make risk intuitive
    fill_opacity=0.7,
    line_opacity=0.2,
    nan_fill_color="pink",  #color doesn't actually matter here because of next code line!
    nan_fill_opacity=0.0,  #prevent folium from filling areas with no data as black
    legend_name="Historic Redlining Indicator",
)

detroit_chloro.color_scale.colors = list(reversed(detroit_chloro.color_scale.colors))
detroit_chloro.add_to(detroit_hri_map)

#visualize data!
detroit_hri_map


In [ ]:
#load race variables
race_vars = {
    "P1_003N": "white",   # white pop
    "P1_004N": "black",   # black pop
    "P1_001N": "total"    # total pop
}

#create race dataframe
race_data = census.pl.get(
    list(race_vars.keys()),
    geo={
        "for": "tract:*", # all tracts
        "in": "state:26 county: 099,125,163" # Michigan state and all counties that include Detroit (Wayne, Oakland, Macomb)
    }
)

race_df = pd.DataFrame(race_data)
race_df = race_df.set_index("tract")
race_df = race_df.rename(columns=race_vars) # gices columns nicer names

# clean up dataframe to change index and match the tract column with the shapefile TRACTCE column
race_df = race_df.reset_index()
race_df = race_df.rename(columns={"tract": "TRACTCE"})

#add a column for percent black population
race_df["pct_black"] = race_df["black"] / race_df["total"]

#merge!
detroit_race_gdf = wayne_tract_gdf.merge(race_df, on="TRACTCE")


#make chloropleth!
ax = detroit_race_gdf.plot(figsize=(10, 10), column="pct_black", cmap="OrRd", legend=True)
ax.axis("off")
ax.set_title("Black Population (%) in Detroit Area")
detroit_outline.boundary.plot(ax=ax, edgecolor="black", linewidth=1.0); # this adds a boundary to visualize the Detroit city boundary

plt.savefig("detroit_race.png")

##**Car Ownership**


In [ ]:
#importing acs variables
detroit_transportation = census.acs5.get(
    (
        # Means of Transportation to Work – Total
        "B08301_001E",
        # Means of Transportation to Work – Total – Public transportation (excluding taxicab) – Subway or elevated rail
        "B08301_011E",
        #Means of transportation to work - Total - Car, truck, or van
        "B08301_002E",
        #public transit- Black or African American
        "B08505B_004E",
        #total - Black or African American
        "B08505B_001E",
        #car truck or van, drove alone - Black or African American
        "B08505B_002E",
        #car truck or van, carpooled - Black or African American
        "B08505B_003E",
        #household by number of vehicles available- total
        "B08201_001E",
        #no vehicle available
        "B08201_002E",
        #1 vehicle available
        "B08201_003E",
        #2 vehicles available
        "B08201_004E",
        #3 vehicles available
        "B08201_005E",
        #4+ vehicles available
        "B08201_006E",
        #total civilian employed population 16+
        "C24030_001E",
        #male finance population
        "C24030_014E",
        #male healthcare
        "C24030_023E",
        #male manufacturing
        "C24030_007E",
        #male tech services
        "C24030_018E",
        #female manufacturing
        "C24030_034E",
        #female finance
        "C24030_041E",
        #female tech services
        "C24030_045E",
        #female healthcare
        "C24030_050E",
        #white population
        "B02001_002E",
        #total pop
        "B02001_001E",
        #median income
        "B19326_001E",
        ),
    geo={
        "for": "tract:*",
        "in": f"state:26 county:099,125,163",
    },
    year=2022,
)

In [ ]:
#renaming acs variable to be understandable
detroit_transportation = pd.DataFrame(detroit_transportation).rename(
    columns={
        "B08301_001E": "commuter_count",
        "B08301_011E": "bus_commuter_count",
        "B08301_002E": "car_commuter_count",
        # "B08505B_004": "black_transit_commuter_count",
        # "B08505B_001E": "black_count",
        # "B08505B_002E": "black_car_commuter_count",
        # "B08505B_003E": "black_carpool_commuter_count",
        # NOTE: these weren't coded correctly for the API and returned columns entirely populated by null values

        "B08105B_004E": "black_transit_commuter_count",
        "B08105B_001E": "black_count",
        "B08105B_002E": "black_car_commuter_count",
        "B08105B_003E": "black_carpool_commuter_count",


        "B08201_001E": "vehicle_count_total",
        "B08201_002E": "no_vehicle_count",
        "B08201_003E": "one_vehicle_count",
        "B08201_004E": "two_vehicle_count",
        "B08201_005E": "three_vehicle_count",
        "B08201_006E": "four_vehicle_count",
        "C24030_001E": "employed_count",
        "C24030_014E": "male_finance_count",
        "C24030_023E": "male_healthcare_count",
        "C24030_007E": "male_manufacturing_count",
        "C24030_018E": "male_tech_services_count",
        "C24030_034E": "female_manufacturing_count",
        "C24030_041E": "female_finance_count",
        "C24030_045E": "female_tech_services_count",
        "C24030_050E": "female_healthcare_count",
        "B02001_002E": "white_count",
        "B02001_001E": "total_count",
        "B08133_001E": "aggregate_commute_time_min",
        "B19326_001E": "median_income",
    }
)

In [ ]:
#new column based on if you have a vehicle in general (summing)
detroit_transportation['have_vehicle'] = detroit_transportation['one_vehicle_count'] + detroit_transportation['two_vehicle_count'] + detroit_transportation['three_vehicle_count'] + detroit_transportation['four_vehicle_count']
#taking have_vehicle and dividing by total population for percent
detroit_transportation['vehicle_pct']=100*detroit_transportation['have_vehicle']/detroit_transportation['vehicle_count_total']
#percent of population without vehicle
detroit_transportation['no_vehicle_pct']=100*detroit_transportation['no_vehicle_count']/detroit_transportation['vehicle_count_total']

In [ ]:
#dividing geoid column to get more granular info
detroit_transportation["GEOID"] = (
    detroit_transportation["state"]
    + detroit_transportation["county"]
    + detroit_transportation["tract"]
)
detroit_transportation = detroit_transportation.set_index("GEOID")

#percent of pop using buses to commute
detroit_transportation["bus_pct"] = 100 * detroit_transportation["bus_commuter_count"] / detroit_transportation["commuter_count"]

In [ ]:
#removing some zero values
detroit_transportation = detroit_transportation[detroit_transportation.commuter_count > 0]

##**Employment**

### ACS plots

In [ ]:
# ---- Step 1: API Query Function ----

def query_sld(fields, state_fips='26'):
    url = "https://geodata.epa.gov/arcgis/rest/services/OA/SmartLocationDatabase/MapServer/1/query"
    params = {
        "where": f"STATEFP='{state_fips}' AND (COUNTYFP='099' OR COUNTYFP='125' OR COUNTYFP='163')",
        "outFields": ",".join(fields),
        "returnGeometry": "false",
        "f": "json"
    }
    response = requests.get(url, params=params)
    features = response.json()['features']
    attributes = [f['attributes'] for f in features]
    return pd.DataFrame(attributes)

# ---- Step 2: Pull the Data ----
# Not including ACS variables, already pulled those

fields_of_interest = {"GEOID20":"GEOID20",
                      "TRACTCE": "tract",
                      #"C_R_White":"% White in County",
                      "D5DRI": "Transit Job Accessibility",
                      "D5CRI": "Car Job Accessibility",

                      "D5DR": "Transit Job Accessibility2",
                      "D5CR": "Car Job Accessibility2",

                      "D4B025": "Proportion of CBG employment within ¼ mile of fixed-guideway transit stop",

                      "E5_Svc": "Service Jobs",
                       "E5_Ind": "Industrial Jobs",
                       "E5_Ret": "Retail Jobs",
                      'E8_Hlth':'Healthcare Jobs',


                      "E_PctLowWage": "% Low Wage Workers",
                      "Pct_AO0": "% Zero-Car Households",

                      'D2R_JOBPOP':'Regional Diversity',

                      'D3AAO':'Car-oriented road networks',
                      'D3APO':'Pedestrian-oriented road networks',

                      'D4A': 'Distance from nearest transit stop (m)'

                      }


In [ ]:
#pulling variables from dataset and saving them
sld_df=query_sld(list(fields_of_interest.keys()))

In [ ]:
#doing away w gross column names
sld_df= sld_df.rename(columns=fields_of_interest)


In [ ]:
#more renaming
sld_df=sld_df.rename(columns={"GEOID20":"GEOID", "tract":"TRACTCE"})



In [ ]:
#joining with detroit gdf to have everything in the same place
detroit_transportation_gdf = detroit_tract_gdf.join(detroit_transportation)


In [ ]:
#adding new dataframe to gdf
detroit_epa_gdf =  sld_df.merge(detroit_transportation_gdf, on = 'TRACTCE', how = 'inner')

In [ ]:
detroit_epa_gdf

In [ ]:
from matplotlib import colors
import matplotlib.cm as cm



# build the ScalarMappable

var1 = 'Transit Job Accessibility'
var2 = 'Car Job Accessibility'
df = gpd.GeoDataFrame(
    detroit_epa_gdf[(detroit_epa_gdf[var1] > -99999.0)],
    geometry='geometry'
)

# pick a common vmin/vmax (or whatever range you like)
vmin = min(df[var1].min(), df[var2].min())
vmax = max(df[var1].max(), df[var2].max())
norm = colors.Normalize(vmin=vmin, vmax=vmax)
sm = cm.ScalarMappable(norm=norm, cmap='viridis')
sm.set_array([])
# create a 1×2 figure, share x & y so both maps use the same extents
fig, (ax1, ax2) = plt.subplots(
    1, 2,
    figsize=(6, 3),
    sharex=True, sharey=True
)

# left: Car
df.plot(
    column=var2,
    cmap='viridis',
    legend=False,
    ax=ax1
)
ax1.set_title("Job Accessibility by Car\n(EPA Index)")

# right: Transit
df.plot(
    column=var1,
    cmap='viridis',
    legend=False,
    ax=ax2
)
ax2.set_title("Job Accessibility by Transit\n(EPA Index)")

# remove all ticks & tick‐labels on both axes
for ax in (ax1, ax2):
    ax.tick_params(
        axis='both',
        which='both',
        bottom=False,    # no x‐ticks
        left=False,      # no y‐ticks
        labelbottom=False,
        labelleft=False
    )

cbar = fig.colorbar(
    sm,
    ax=[ax1, ax2],
    orientation='vertical',
    fraction=0.05,
    pad=0.06
)
cbar.set_label("EPA Index", rotation=90, labelpad=9)

plt.show()

In [ ]:
var ='Distance from nearest transit stop (m)'

df_no_outliers = detroit_epa_gdf[(detroit_epa_gdf[f"{var}"] >min(detroit_epa_gdf[f"{var}"]))]
df_no_outliers = detroit_epa_gdf[(detroit_epa_gdf[f"{var}"] > -99999.0) ]
df_no_outliers = gpd.GeoDataFrame(df_no_outliers, geometry='geometry')
ax = df_no_outliers.plot(figsize=(4, 4), column=f"{var}", legend=True, cmap='viridis', legend_kwds={'pad':0.2})
ax.axis("off")
ax.set_title(f"Distance from nearest transit stop (m)")
plt.show()

<div>
<img src='https://drive.google.com/uc?id=1dwnmoKiP2_fghBPzMIOwEFUIfTm3OyS2' width="300"> <img src='https://drive.google.com/uc?id=1Tm51ygOA-G0wEEsrfYF3vIS3EFI345nE' width="300">
</div>
<div>
<img src='https://drive.google.com/uc?id=11Fg703FoEuH-vDHOZA2NEZv22XlvyNSq' width="300">
</div>

<div>
<img src='https://drive.google.com/uc?id=1dwnmoKiP2_fghBPzMIOwEFUIfTm3OyS2' width="300">
</div>

In [ ]:
# Define the bins
bins = [0, 0.25, 0.50, 0.75, 1]

var1 = 'Car Job Accessibility'
var2 = '% Zero-Car Households'
print(min(detroit_epa_gdf[f"{var1}"]))
print(min(detroit_epa_gdf[f"{var2}"]))

print(min(detroit_epa_gdf[f'{var1}']))

# # Bin the first variable

detroit_epa_gdf[f'{var1}_class'] = pd.qcut(detroit_epa_gdf[f'{var1}'], q=4, labels = np.arange(1, int(len(bins))).astype(str))

detroit_epa_gdf[f'{var1}_class'] = detroit_epa_gdf[f'{var1}_class'].astype('str')

# Bin the second variable
detroit_epa_gdf[f'{var2}_class'] = pd.qcut(detroit_epa_gdf[f'{var2}'], q= 4, labels = np.arange(1, int(len(bins))).astype(str))

#pd.qcut(detroit_epa_gdf[f'{var2}'], q=4, labels = np.arange(1, int(len(bins))).astype(str))
detroit_epa_gdf[f'{var2}_class'] = detroit_epa_gdf[f'{var2}_class'] = detroit_epa_gdf[f'{var2}_class'].astype('str')



In [ ]:
pd.cut(detroit_epa_gdf[f'{var2}'], bins= 5, labels = np.arange(1, int(len(bins)+1)).astype(str)) #.value_counts()

#detroit_epa_gdf[f'{var1}']

In [ ]:
# Code created x bins to 1, 2, 3
x_class_codes = np.arange(1, len(bins))
d = dict(zip(detroit_epa_gdf[f'{var1}_class'].value_counts().sort_index().index, x_class_codes))
detroit_epa_gdf[f'{var1}_class'] = detroit_epa_gdf[f'{var1}_class'].replace(d)

# Code created y bins to A, B, C
y_class_codes = np.arange(1, len(bins))
d = dict(zip(detroit_epa_gdf[f'{var2}_class'].value_counts().sort_index().index, y_class_codes))
detroit_epa_gdf[f'{var2}_class'] = detroit_epa_gdf[f'{var2}_class'].replace(d)

# Combine x and y codes to create Bi_Class
detroit_epa_gdf['Bi_Class'] = detroit_epa_gdf[f'{var1}_class'].astype('str') + detroit_epa_gdf[f'{var2}_class'].astype('str')

In [ ]:
detroit_epa_gdf[f'{var2}_class'].value_counts()

In [ ]:
detroit_epa_gdf.columns

In [ ]:
#from matplotlib import colors
hex_colors = [
    "#fffbf5", "#fce4f3", "#f0b9e1", "#d889c4",
    "#e4fbf2", "#d7dae8", "#c2acd0", "#a376ad",
    "#c4f5ed", "#b3cede", "#969abe", "#6e6091",
    "#a1ede6", "#8abfd1", "#6e88ad", "#41456d"
]
cmap  = colors.ListedColormap(hex_colors)

In [ ]:
from matplotlib.patches import Patch
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import to_rgb
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

var = 'Bi_Class'
df_no_outliers = detroit_epa_gdf #[(detroit_epa_gdf[f"{var}"] > -99999.0) ]
df_no_outliers = gpd.GeoDataFrame(df_no_outliers, geometry='geometry')
ax = df_no_outliers.plot(figsize=(4, 4), column=f"{var}", legend=False, cmap = cmap) # cmap='viridis')
ax.axis("off")
ax.set_title(f"Spatial Mismatch: \nDisparities in Car Ownership \n Enforce Inequitable Job Access")

# Convert to an (4,4,3) RGB array
rgb = np.array([to_rgb(c) for c in hex_colors])
grid = rgb.reshape((4, 4, 3))

# now inset the 4×4 legend at upper right:
ax_leg = inset_axes(ax,
                    width="25%", height="25%",
                    loc='lower right',
                    bbox_to_anchor=(.3, 0.1, 1, 1.2),      # x, y in axes-fraction coordinates
    bbox_transform=ax.transAxes,
                    borderpad=0)

ax_leg.imshow(grid, origin='lower', extent=[0,100,0,100])
ax_leg.set_xticks([0,25,50,75,100])
ax_leg.set_yticks([0,25,50,75,100])
ax_leg.tick_params(left=False, bottom=False, labelsize=8)
ax_leg.set_ylabel("% zero-car \n households →", fontsize=8, labelpad=2)
ax_leg.set_xlabel("↑ accessibility of \njob sites (by car)", fontsize=8, labelpad=2)
ax_leg.spines[:].set_visible(False)
plt.show()

For this part of the project, We wanted to show where the major car companies in Detroit are located, and how big they are based on how many people they employ. We used data that lists different motor businesses and filtered it down to only Ford, GM, and FCA, the companies that are usually called the "Detroit Three." We removed any companies that had zero employees, because the goal was to highlight where the jobs really are.
To visualize this,we  created a map of Detroit using Folium. We also used colored circles to show where each company’s site is located. The size of each circle depends on how many local employees work there, so the bigger the dot, the more people that company employs at that spot. Each company has its own color GM is turquoise, FCA is lemonchiffon, and Ford is lavender. We also added a popup to each dot, so when you click on it, you can see the name of the company and how many employees they have there.
We included a legend in the bottom corner of the map so that it's easy to tell which color stands for which company. This map is part of a larger project we are working on about transportation, economic geography, and inequality in Detroit. By showing where these jobs are concentrated, it helps us to explain patterns of access, mobility, and even health in different parts of the city. It also gives a clear visual of how these big companies are spread across Detroit and how their presence might influence nearby communities.


In [ ]:
# Let us filter only GM, FCA, and FORD with more than 0 employees
d3 = motor_businesses[
    (motor_businesses['D3'].isin(['GM', 'FCA', 'FORD'])) &
    (motor_businesses['Total Employees'] > 0)
]

# Fill missing D3 values with 'Other'
motor_businesses['D3'] = motor_businesses['D3'].fillna('Other')

# Define custom colors for each company
corp = {'GM': 'turquoise', 'FCA': 'lemonchiffon', 'FORD': 'lavender'}

# Let us make the map and center it on Detroit
detroit = folium.Map(location=[42.3314, -83.0468], zoom_start=9, tiles="CARTODB darkmatter")

# Now we plot the companies with bubbles — size shows employee count
for _, site in d3.iterrows():
    lat, lon = site['Latitude'], site['Longitude']
    company = site['D3']
    folium.CircleMarker(
        location=[lat, lon],
        radius=2 * np.log(site['Local Employees']),  # circle size depends on employees
        fill=True,
        fill_opacity=0.5,
        opacity=1,
        color=corp[company],
        popup=f"{company}, {int(site['Local Employees'])} employees"  # show info when clicked
    ).add_to(detroit)

# Let us add the legend to explain what the colors mean
legend_html = """
<div style="position: fixed;
     bottom: 50px; left: 50px; width: 180px; height: 120px;
     border:2px solid white; z-index:9999; font-size:14px;
     background-color:grey; opacity: 0.8; color:white;
     padding: 10px;">
     <b>Legend</b><br>
     <i style="background:turquoise; width:10px; height:10px; float:left; margin-right:5px;"></i> GM<br>
     <i style="background:lemonchiffon; width:10px; height:10px; float:left; margin-right:5px;"></i> FCA<br>
     <i style="background:lavender; width:10px; height:10px; float:left; margin-right:5px;"></i> FORD
</div>
"""
detroit.get_root().html.add_child(folium.Element(legend_html))

# Show the map
img_data=detroit._to_png(5)
img=Image.open(io.BytesIO(img_data))
img.save('detroit_map.png')
detroit


### Transportation Method Comparison: Driving vs Bus

2 Key Employment Centers:


1.   GM Tech Center, Warren, MI -> Suburban
2.   Cadillac Place, Detroit, MI -> Urban



In [ ]:
# Loosely based on Week 9 notebook
# Init POIs and parameters (derived from motor_businesses)
locations = [
    {"name": "GM Tech Center", "lat": 42.508552, "lon": -83.039437},
    {"name": "GM Cadillac Place",  "lat": 42.368748, "lon": -83.075383},
]

# Only car and bus now
modes = ["car", "bus"]
speeds = {"car": 2640, "bus": 1144}  # ft per minute
trip_times = [15, 30, 60] # minutes
iso_colors = ox.plot.get_colors(n=len(trip_times), cmap="plasma", start=0)

# Download & project ONE drive graph per POI
buffer_dist = 20000  # meters
graphs = {}
for loc in locations:
    G_drive = ox.graph_from_point(
        (loc["lat"], loc["lon"]),
        dist=buffer_dist,
        network_type="drive"
    )
    G_drive = ox.project_graph(G_drive, to_crs="epsg:2253")
    # Compute length ONCE
    for u, v, k, data in G_drive.edges(keys=True, data=True):
        data["length_ft"] = data["length"] * 3.28084
    graphs[loc["name"]] = G_drive

# 2×2 grid (2 POIs × 2 modes)
fig, axes = plt.subplots(2, 2, figsize=(16, 12), subplot_kw={"xticks": [], "yticks": []})
buffer_ft = buffer_dist * 3.28084

for i, loc in enumerate(locations):
    G = graphs[loc["name"]]
    # Project POI ONCE
    poi = (
        gpd.GeoSeries([Point(loc["lon"], loc["lat"])], crs="EPSG:4326")
           .to_crs(G.graph["crs"])
           .iloc[0]
    )
    ego = ox.nearest_nodes(G, X=[poi.x], Y=[poi.y])[0]

    for j, mode in enumerate(modes):
        ax = axes[i][j]
        spd = speeds[mode]

        # Time BEFORE building the subgraph
        for u, v, k, data in G.edges(keys=True, data=True):
            data["time"] = data["length_ft"] / spd

        # Ego‐graph & compute timed distances (Shoutout Dijkstra/2110)
        max_t = max(trip_times)
        big_sub = nx.ego_graph(G, ego, radius=max_t, distance="time")
        dist = nx.single_source_dijkstra_path_length(big_sub, ego, weight="time")

        # Convex‐hull isochrones based on that dist
        hulls = {}
        for t in trip_times:
            pts = [
                Point((big_sub.nodes[n]["x"], big_sub.nodes[n]["y"]))
                for n, d in dist.items() if d <= t
            ]
            gs = gpd.GeoSeries(pts, crs=G.graph["crs"])
            hulls[t] = gs.union_all().convex_hull # Is union_all really the way?

        # Plot bands
        alpha = 0.4  # car/bus both semi‐transparent
        for t, c in zip(sorted(trip_times, reverse=True), iso_colors[::-1]):
            gpd.GeoSeries([hulls[t]], crs=G.graph["crs"]) \
               .plot(ax=ax, color=c, alpha=alpha, linewidth=0)

        # Roads on top
        ox.plot_graph(
            G, ax=ax, show=False, close=False,
            edge_color="#333333", edge_alpha=0.8,
            edge_linewidth=0.5, node_size=0
        )

        # Mark & center
        ax.scatter(poi.x, poi.y, marker="*", color="black", s=150)
        ax.set_xlim(poi.x - buffer_ft, poi.x + buffer_ft)
        ax.set_ylim(poi.y - buffer_ft, poi.y + buffer_ft)

        ax.set_title(f"{loc['name']}\n({mode.capitalize()})", fontsize=14)
        ax.set_axis_off()

# Legend
handles = [
    mlines.Line2D([], [], marker="s", color=c, linestyle="None", markersize=10)
    for c in iso_colors
]
labels = [f"{t} min" for t in trip_times]
fig.legend(
    handles, labels,
    title="Travel-time isochrones",
    ncol=len(trip_times),
    loc="lower center",
    fontsize=12, title_fontsize=13
)

plt.tight_layout(rect=[0, 0.05, 1, 1])
plt.show()

## Regression

In [ ]:
#Here we're summing together the ACS variables we pulled on industries to create our y-values

#Creating a total manufacturing column with the male and female counts
detroit_transportation['manufacturing_count']=detroit_transportation['male_manufacturing_count']+detroit_transportation['female_manufacturing_count']

#Creating a total healthcare column with the male and female counts
detroit_transportation['healthcare_count']=detroit_transportation['male_healthcare_count']+detroit_transportation['female_healthcare_count']

#Creating a total finance column with the male and female counts
detroit_transportation['finance_count']=detroit_transportation['male_finance_count']+detroit_transportation['female_finance_count']

#Creating a total tech services column with the male and female counts
detroit_transportation['tech_services_count']=detroit_transportation['male_tech_services_count']+detroit_transportation['female_tech_services_count']

#Creating a total industry column with the male and female counts
detroit_transportation['industry_count']=detroit_transportation['manufacturing_count']+detroit_transportation['healthcare_count']+detroit_transportation['finance_count']+detroit_transportation['tech_services_count']

#Pct of working population in these industries
detroit_transportation['industry_pct']=100*(detroit_transportation['industry_count']/detroit_transportation['employed_count'])

#Pct white population
detroit_transportation['white_pct']=100*(detroit_transportation['white_count']/detroit_transportation['total_count'])

detroit_transportation=detroit_transportation[detroit_transportation['median_income']>0]


In [ ]:
#Creating a new df for the regression with just our x and y variables + indentifing data. Dropping any nan values
regression_df = detroit_transportation[['vehicle_pct', 'bus_pct', 'white_pct', 'industry_pct', 'median_income']].dropna()

Before training a multivariate regression, we first created a multi-dimensional scattplot to illustrate the relationship of the percent of population with vehicles, percent of population using the bus for commutes, percent of population that is white alone, and the percent of the population that is working within the top Detroit industries.

In [ ]:
#Creating a 3-dimensional scatter plot to illustrate relationship of regression variables.

fig=plt.figure(figsize=(10,7))
ax=fig.add_subplot(111, projection='3d')

#Variables to be used:
x=regression_df['vehicle_pct']
y=regression_df['industry_pct']
z=regression_df['bus_pct']
c=regression_df['white_pct']

#This plot uses a colorbar to show the fourth dimension/variable
graph= ax.scatter(x, y, z, c=c, cmap='magma')

#Labeling each axis (and colorbar)
ax.set_xlabel('Population with a vehicle %')
ax.set_ylabel('Employed in top industries %')
ax.set_zlabel('Commute using bus %')
fig.colorbar(graph, label='White population %', pad=0.15)
ax.zaxis.labelpad=2
plt.title('Multivariable Transportation Analysis')
plt.savefig('regression1.png')
plt.show()

In [ ]:
#Training our multivariate regression
#Given input list (x variables)
input_vars = ["vehicle_pct", "bus_pct", "white_pct"]

#Fitting variables to a linear regression
X=regression_df[input_vars]
y=regression_df['industry_pct']
model=LinearRegression().fit(X,y)

#Printing the coefficients for each x-variable
for var_name, var_coef in zip(input_vars, model.coef_):
    print(var_name+': '+str(round(var_coef, 2)))

#Printing the intercept of the model
print('intercept: '+str(round(model.intercept_, 2)))

In [ ]:
#Creating a 3-dimensional scatter plot to illustrate relationship of regression variables.

fig=plt.figure(figsize=(10,7))
ax=fig.add_subplot(111, projection='3d')

#Variables to be used:
x=regression_df['vehicle_pct']
y=regression_df['median_income']
z=regression_df['bus_pct']
c=regression_df['white_pct']
#This plot uses a colorbar to show the fourth dimension/variable
graph= ax.scatter(x, y, z, c=c, cmap="magma")

#Labeling each axis (and colorbar)
ax.set_xlabel('Population with a vehicle %')
ax.set_ylabel('Median income ($)')
ax.set_zlabel('Commute using bus %')
fig.colorbar(graph, label='White population %', pad=0.15)
ax.zaxis.labelpad=2
ax.set_yticks(np.arange(10000, y.max(), 10000))
plt.title('Multivariable Transportation Analysis')
plt.savefig('regression2.png')
plt.show()

In [ ]:
#Training our multivariate regression

#Fitting variables to a linear regression
X=regression_df[input_vars]
y=regression_df['median_income']
model=LinearRegression().fit(X,y)

#Printing the coefficients for each x-variable
for var_name, var_coef in zip(input_vars, model.coef_):
    print(var_name+': '+str(round(var_coef, 2)))

#Printing the intercept of the model
print('intercept: '+str(round(model.intercept_, 2)))